## 场内基金套利策略
### 简介：通过对场内LOF基金进行监测，发现其是否与场外净值有较大偏差来进行套利
### 操作方式：
1.查看场外LOF基金前一日净值与当日实时估算净值  
2.查看场内该基金的实时交易价格  
3.判断实时交易价格是否偏离实时估算净值大于100bps  
4.判断交易量是否较前几日有明显放大  
5.在首先满足步骤3的条件下，发送信号。

### 一期计划：
实现信号功能：能够较为准确的发现套利机会，程序能够在交易时间段内持续监控  
可改进项：通过机器学习多分类来解决识别误差问题

### 二期计划：
实现信息发送功能：发送至微信或者邮件中

### 三期计划：
实现自动化交易功能。



In [1]:
###1.连接各类数据接口

In [20]:
import math
import requests,json,time
import easyquotation as eq
import numpy as np
import pandas as pd
import sqlalchemy as sql
import json

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)

#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
class fundFeatures():
    def __init__(self, code, amplitude,turnover,*args):
        self.code = code
        self.amplitude = amplitude
        self.turnover = turnover
        self.esval,self.gain,self.pre_price = self.get_estimated_netval()
        self.amplitude,self.turnover,self.now,self.bid1,self.bid1_volume,self.bid2,self.bid2_volume,self.ask1,self.ask1_volume,self.ask2,self.ask2_volume=self.get_now_data()
        
    def change(self):
        self.esval,self.gain,self.pre_price = self.get_estimated_netval()
        self.amplitude,self.turnover,self.now,self.bid1,self.bid1_volume,self.bid2,self.bid2_volume,self.ask1,self.ask1_volume,self.ask2,self.ask2_volume=self.get_now_data()
        
    def get_estimated_netval(self):
        t = time.time()
        timestamp = round(t * 1000)
        Headers = {'content-type':'application/json','User-Agent': 'Apache-HttpClient/4.5.2 (Java/1.8.0_102)'}
        TTurl = "http://fundgz.1234567.com.cn/js/" + str(self.code) +  ".js?rt=" + str(timestamp)
        r = requests.get(TTurl, headers=Headers)
        GetMsg = r.text
        if "fundcode" in GetMsg:
            estimated_netval = GetMsg.split(",")[4].split(":")[1].replace('"','')
            gain = GetMsg.split(",")[5].split(":")[1].replace('"','')
            pre_price=  GetMsg.split(",")[3].split(":")[1].replace('"','')
            return float(estimated_netval),float(gain),float(pre_price)
        else:
            return 0,0,0
    def get_now_data(self):
        code = str(self.code)
        quotation =eq.use('tencent')
        fund_dict =quotation.real(code)
        return fund_dict[code]['振幅'],fund_dict[code]['成交额(万)'],fund_dict[code]['now'],fund_dict[code]['bid1'],fund_dict[code]['bid1_volume'],fund_dict[code]['bid2'],fund_dict[code]['bid2_volume'],fund_dict[code]['ask1'],fund_dict[code]['ask1_volume'],fund_dict[code]['ask2'],fund_dict[code]['ask2_volume']
        
    def as_dict(self):
        return {'code': self.code, 'amplitude': self.amplitude, 'turnover': self.turnover,'gain': self.gain,'pre_price': self.pre_price,
               'es_value': self.esval,'now': self.now,'bid1': self.bid1,'bid1_volume': self.bid1_volume,'bid2': self.bid2,'bid2_volume': self.bid2_volume,'ask1': self.ask1,'ask1_volume': self.ask1_volume,'ask2': self.ask2,'ask2_volume':self.ask2_volume}

In [4]:
def initFunds():
    #读取fof基金列表
    df=pd.read_csv('/Users/zhangming/Work/invest/lof.csv')
    tmp_funds_list = df['main_code'].to_list()
    str_funds_list= [str(x) for x in tmp_funds_list]
    #对基金进行选择操作，逻辑成交额大于10万，并且振幅大于3个点
    quotation = eq.use('tencent') # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
    funds_dict = quotation.stocks(str_funds_list)
    valid_funds = []
    #初始化基金数据，获取实时涨跌，振幅，并转为pd格式
    for key,value in funds_dict.items():
        obj = fundFeatures(key,value['振幅'],value['成交额(万)'])
        valid_funds.append(obj)
    return valid_funds
#valid_funds = sorted(valid_funds, key=lambda valid_fund: valid_fund.amplitude,reverse=True)

In [7]:
#更新基金实时变动
def get_now_funds(valid_funds):
    for item in valid_funds:
        item.change()
    real_fund = pd.DataFrame([x.as_dict() for x in valid_funds])
    real_fund = real_fund.drop(real_fund[real_fund.es_value == 0 ].index)
    real_fund=real_fund[(real_fund['turnover'] > 0 )|(real_fund['bid1']>0)]
    real_fund['change_pct'] = (real_fund['now']-real_fund['es_value'])/real_fund['es_value'] *100
#     crtieria = real_fund.sort_values(by=['change_pct','amplitude'],ascending=[False,False])
#     crtieria.reset_index(drop=True, inplace=True)
    return real_fund

    

In [17]:
valid_funds = initFunds()

#167508,161912,160526

In [18]:
#获得实时基金变动
res = get_now_funds(valid_funds)

In [19]:

res[(res['turnover']>=50000)&(res['turnover']<=3000000)].sort_values(by=['change_pct','amplitude'],ascending=[True,False])

,code,amplitude,turnover,gain,pre_price,es_value,now,bid1,bid1_volume,bid2,bid2_volume,ask1,ask1_volume,ask2,ask2_volume,change_pct
35,160322,4.18,1710000.0,3.03,1.5212,1.5673,1.536,1.536,1400,1.535,22000,1.538,2700,1.539,18700,-1.997065
75,163110,7.45,170000.0,0.36,2.0334,2.0407,2.014,2.014,200,2.013,500,2.100,200,2.187,400,-1.308375
168,161030,6.43,1020000.0,-1.69,0.9340,0.9182,0.909,0.909,15900,0.908,93100,0.916,11400,0.932,21100,-1.001960
254,501089,2.05,610000.0,-0.03,1.2535,1.2531,1.242,1.240,9000,1.235,300,1.249,52000,1.258,5400,-0.885803
163,161025,1.92,390000.0,0.25,0.8890,0.8912,0.884,0.884,22700,0.882,3500,0.888,700,0.890,100,-0.807899
152,160635,1.92,190000.0,1.65,1.4030,1.4262,1.415,1.416,600,1.415,1600,1.429,6000,1.430,2700,-0.785304
41,160621,0.36,60000.0,-0.16,1.3940,1.3918,1.381,1.380,1200,1.379,29700,1.397,2000,1.398,20000,-0.775974
147,160630,3.46,1900000.0,-0.45,1.0730,1.0682,1.060,1.059,14000,1.058,1500,1.060,8400,1.061,4300,-0.767647
146,160629,1.98,120000.0,-1.54,1.0130,0.9974,0.990,0.990,2500,0.981,4000,0.994,10700,1.002,0,-0.741929
60,161217,3.02,90000.0,-0.43,0.9610,0.9569,0.950,0.948,6100,0.946,100,0.950,2900,0.960,3300,-0.721078


In [20]:
now_date = time.strftime('%Y-%m-%d',time.localtime())
res['date']=now_date
file_path='/Users/zhangming/Work/invest/'+now_date+'.csv'
res.to_csv(file_path)

In [247]:
if __name__ == '__main__':
    #初始化基金
    valid_funds = initFunds()
    
    #获得实时基金变动
    res = get_now_funds(valid_funds)
    res


In [1]:
res = get_now_funds(valid_funds)
res

NameError: name 'get_now_funds' is not defined

In [ ]:
for item in valid_funds:
    item.change()
real_fund = pd.DataFrame([x.as_dict() for x in valid_funds])
real_fund = real_fund.sort_values(by=['amplitude','turnover'],ascending=[False,False])
real_fund = real_fund.drop(real_fund[real_fund.es_value == 0 ].index)
real_fund=real_fund[(real_fund['turnover'] > 0 )|(real_fund['bid1']>0)]
real_fund.reset_index(drop=True, inplace=True)
real_fund['change_pct'] = (real_fund['now']-real_fund['es_value'])/real_fund['es_value'] *100



In [54]:
quotation = eq.use('tencent') # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
quotation.stocks(['161030', '160516'])
#选相应波动基金
for 
#tmp_dict = quotation.market_snapshot(prefix=True) # prefix 参数指定返回的行情字典中的股票代码 key 是否带 sz/sh 前缀

{'161030': {'name': '体育',
  'code': '161030',
  'now': 1.038,
  'close': 1.045,
  'open': 0.941,
  'volume': 933200.0,
  'bid_volume': 330100,
  'ask_volume': 603100.0,
  'bid1': 1.038,
  'bid1_volume': 38000,
  'bid2': 1.035,
  'bid2_volume': 5000,
  'bid3': 1.014,
  'bid3_volume': 10000,
  'bid4': 0.964,
  'bid4_volume': 3700,
  'bid5': 0.963,
  'bid5_volume': 11000,
  'ask1': 1.04,
  'ask1_volume': 10100,
  'ask2': 1.042,
  'ask2_volume': 105200,
  'ask3': 1.045,
  'ask3_volume': 18600,
  'ask4': 1.08,
  'ask4_volume': 10100,
  'ask5': 1.09,
  'ask5_volume': 1000,
  '最近逐笔成交': '',
  'datetime': datetime.datetime(2021, 3, 5, 16, 15, 3),
  '涨跌': -0.007,
  '涨跌(%)': -0.67,
  'high': 1.042,
  'low': 0.941,
  '价格/成交量(手)/成交额': '1.038/9332/897730',
  '成交量(手)': 933200,
  '成交额(万)': 900000.0,
  'turnover': None,
  'PE': None,
  'unknown': '',
  'high_2': 1.042,
  'low_2': 0.941,
  '振幅': 9.67,
  '流通市值': None,
  '总市值': None,
  'PB': 0.0,
  '涨停价': 1.15,
  '跌停价': 0.941,
  '量比': 1.93,
  '委差': -773.0

In [ ]:

# def GetFundData(fundId,GetTimestamp):
#     Headers = {'content-type':'application/json','User-Agent': 'Apache-HttpClient/4.5.2 (Java/1.8.0_102)'}
#     TTurl = "http://fundgz.1234567.com.cn/js/" + str(fundId) +  ".js?rt=" + str(GetTimestamp)
#     r = requests.get(TTurl, headers=Headers)
#     GetMsg = r.text
#     print(GetMsg)
#     if "fundcode" in GetMsg:
#         FundID = GetMsg.split(",")[0].split(":")[1]
#         FundName = GetMsg.split(",")[1].split(":")[1]
        
#         Gain = GetMsg.split(",")[5].split(":")[1]
#         # print(GetMsg.split(",")[6].split("}")[0])
#         currentTime = GetMsg.split(",")[6].split("}")[0].split("\"gztime\":")[1]
#         # GetData = "基金名称:"+ FundName + "ID" + FundID + "涨跌幅:" + Gain + "当前时间:" + currentTime
#         # GetData =  FundName + " " + FundID + " " + Gain
#         # GetData = "基金名称:"+ FundName + "涨跌幅:" + Gain + "当前时间:" + currentTime
#         GetData = "基金名称:"+ FundName + " "+ "涨跌幅:" + Gain + " "+ "当前时间:" + currentTime
#     return GetData
# #获取时间戳
# def GetTime():
#     t = time.time()
#     Timestamp = round(t * 1000)
#     # print(Timestamp)
#     return Timestamp
# GetTimestamp = GetTime()
# GetFundData('161030',GetTimestamp)